# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 15 2022 8:30AM,241917,10,8274558,Yusen – 3D Matrix,Released
1,Jul 15 2022 8:30AM,241917,10,8275065,Yusen – 3D Matrix,Released
2,Jul 15 2022 8:30AM,241917,10,8275069,Yusen – 3D Matrix,Released
3,Jul 15 2022 8:30AM,241917,10,8275070,Yusen – 3D Matrix,Released
4,Jul 15 2022 8:30AM,241917,10,8275072,Yusen – 3D Matrix,Released
5,Jul 14 2022 3:32PM,241919,16,8274529,Sartorius Bioprocess Solutions,Released
6,Jul 14 2022 3:27PM,241908,10,8274556,Eywa Pharma Inc.,Released
7,Jul 14 2022 3:27PM,241908,10,8274557-SP,Eywa Pharma Inc.,Released
8,Jul 14 2022 3:21PM,241916,16,EMERSA-8275067,"Emersa Waterbox, LLC",Released
9,Jul 14 2022 3:08PM,241915,10,0085873746,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,241914,Released,8
27,241915,Released,27
28,241916,Released,1
29,241917,Released,5
30,241919,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241914,NaN,8.0
241915,NaN,27.0
241916,NaN,1.0
241917,NaN,5.0
241919,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241827,0.0,2.0
241828,0.0,1.0
241836,0.0,7.0
241850,0.0,3.0
241858,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241827,0,2
241828,0,1
241836,0,7
241850,0,3
241858,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241827,0,2
1,241828,0,1
2,241836,0,7
3,241850,0,3
4,241858,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241827,,2
1,241828,,1
2,241836,,7
3,241850,,3
4,241858,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 15 2022 8:30AM,241917,10,Yusen – 3D Matrix
5,Jul 14 2022 3:32PM,241919,16,Sartorius Bioprocess Solutions
6,Jul 14 2022 3:27PM,241908,10,Eywa Pharma Inc.
8,Jul 14 2022 3:21PM,241916,16,"Emersa Waterbox, LLC"
9,Jul 14 2022 3:08PM,241915,10,ISDIN Corporation
36,Jul 14 2022 3:08PM,241914,10,ISDIN Corporation
44,Jul 14 2022 3:03PM,241912,20,Sartorius Stedim North America
45,Jul 14 2022 3:03PM,241911,10,ISDIN Corporation
51,Jul 14 2022 3:01PM,241910,10,"Methapharm, Inc."
70,Jul 14 2022 2:59PM,241909,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 15 2022 8:30AM,241917,10,Yusen – 3D Matrix,,5
1,Jul 14 2022 3:32PM,241919,16,Sartorius Bioprocess Solutions,,1
2,Jul 14 2022 3:27PM,241908,10,Eywa Pharma Inc.,,2
3,Jul 14 2022 3:21PM,241916,16,"Emersa Waterbox, LLC",,1
4,Jul 14 2022 3:08PM,241915,10,ISDIN Corporation,,27
5,Jul 14 2022 3:08PM,241914,10,ISDIN Corporation,,8
6,Jul 14 2022 3:03PM,241912,20,Sartorius Stedim North America,1,
7,Jul 14 2022 3:03PM,241911,10,ISDIN Corporation,,6
8,Jul 14 2022 3:01PM,241910,10,"Methapharm, Inc.",,19
9,Jul 14 2022 2:59PM,241909,10,"Methapharm, Inc.",,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 8:30AM,241917,10,Yusen – 3D Matrix,5,
1,Jul 14 2022 3:32PM,241919,16,Sartorius Bioprocess Solutions,1,
2,Jul 14 2022 3:27PM,241908,10,Eywa Pharma Inc.,2,
3,Jul 14 2022 3:21PM,241916,16,"Emersa Waterbox, LLC",1,
4,Jul 14 2022 3:08PM,241915,10,ISDIN Corporation,27,
5,Jul 14 2022 3:08PM,241914,10,ISDIN Corporation,8,
6,Jul 14 2022 3:03PM,241912,20,Sartorius Stedim North America,,1
7,Jul 14 2022 3:03PM,241911,10,ISDIN Corporation,6,
8,Jul 14 2022 3:01PM,241910,10,"Methapharm, Inc.",19,
9,Jul 14 2022 2:59PM,241909,10,"Methapharm, Inc.",3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 8:30AM,241917,10,Yusen – 3D Matrix,5,
1,Jul 14 2022 3:32PM,241919,16,Sartorius Bioprocess Solutions,1,
2,Jul 14 2022 3:27PM,241908,10,Eywa Pharma Inc.,2,
3,Jul 14 2022 3:21PM,241916,16,"Emersa Waterbox, LLC",1,
4,Jul 14 2022 3:08PM,241915,10,ISDIN Corporation,27,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 8:30AM,241917,10,Yusen – 3D Matrix,5.0,NaN
1,Jul 14 2022 3:32PM,241919,16,Sartorius Bioprocess Solutions,1.0,NaN
2,Jul 14 2022 3:27PM,241908,10,Eywa Pharma Inc.,2.0,NaN
3,Jul 14 2022 3:21PM,241916,16,"Emersa Waterbox, LLC",1.0,NaN
4,Jul 14 2022 3:08PM,241915,10,ISDIN Corporation,27.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 15 2022 8:30AM,241917,10,Yusen – 3D Matrix,5.0,0.0
1,Jul 14 2022 3:32PM,241919,16,Sartorius Bioprocess Solutions,1.0,0.0
2,Jul 14 2022 3:27PM,241908,10,Eywa Pharma Inc.,2.0,0.0
3,Jul 14 2022 3:21PM,241916,16,"Emersa Waterbox, LLC",1.0,0.0
4,Jul 14 2022 3:08PM,241915,10,ISDIN Corporation,27.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3144616,96.0,5.0
12,241905,2.0,0.0
16,967553,5.0,0.0
19,241858,1.0,0.0
20,725664,4.0,1.0
21,1451298,5.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3144616,96.0,5.0
1,12,241905,2.0,0.0
2,16,967553,5.0,0.0
3,19,241858,1.0,0.0
4,20,725664,4.0,1.0
5,21,1451298,5.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,96.0,5.0
1,12,2.0,0.0
2,16,5.0,0.0
3,19,1.0,0.0
4,20,4.0,1.0
5,21,5.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,12,Released,2.0
2,16,Released,5.0
3,19,Released,1.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,5.0,0.0,0.0,0.0,1.0,1.0
Released,96.0,2.0,5.0,1.0,4.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,5.0,0.0,0.0,0.0,1.0,1.0
1,Released,96.0,2.0,5.0,1.0,4.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,5.0,0.0,0.0,0.0,1.0,1.0
1,Released,96.0,2.0,5.0,1.0,4.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()